In [1]:
import sys, os, glob, pickle, toml, json
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.cm as cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def set_background(ax):
    ax.set_extent([-121, -72, 25, 50])

    # Background boundaries
    # ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS)
    states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
    ax.add_feature(states_provinces, edgecolor='gray')
    
    return ax


def cal_ratio_within_bins(data, bins):
    binsc = np.zeros(len(bins)-1)
    ratio = np.zeros(len(bins)-1)
    for i in range(len(bins)-1):
        indi = (data>bins[i]) & (data<=bins[i+1])
        ratio[i] = np.sum(indi)/np.sum(~np.isnan(data))
        binsc[i] = (bins[i]+bins[i+1])/2
    return binsc, ratio

In [2]:
# load info
infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
df_info = pd.read_csv(infile_basin_info)

In [5]:
# input/out and iteration information
iternum = 5
inpath_moasmo = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/'
outpath = f'/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/LargeSampleEmulator_exps_out'

In [9]:
    df_lse_metric = []

    for tarbasin in range(len(df_info)):
    # for tarbasin in range(5):
        for i in range(iternum):
            infile = f'{inpath_moasmo}/level1_{tarbasin}_MOASMOcalib/ctsm_outputs_emutest/iter{i}_many_metric.csv'    
            dfi = pd.read_csv(infile)
            dfi['basin'] = tarbasin
            dfi['iter'] = i
            dfi['trial'] = np.arange(len(dfi))
            if len(df_lse_metric) == 0:
                df_lse_metric = dfi
            else:
                df_lse_metric = pd.concat([df_lse_metric, dfi])

In [10]:
metname = 'kge'
met_stats_lse = np.nan * np.zeros([len(df_info), iternum, 3]) # best; mean; median
for i in range(iternum):
    indi = (df_lse_metric['iter']==i)
    for j in range(len(df_info)):
        indij = indi & (df_lse_metric['basin']==j)
        dij = df_lse_metric[indij][metname].values

        met_stats_lse[j, i, 0] = np.nanmax(dij)
        met_stats_lse[j, i, 1] = np.nanmean(dij)
        met_stats_lse[j, i, 2] = np.nanmedian(dij)


In [14]:
np.nanmedian(met_stats_lse[:,:,0], axis=0)

array([0.46116677, 0.50478524, 0.51495102, 0.52538579, 0.53496219])

In [16]:
import xarray as xr
file = '/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/sfcdata/surfdata_CAMELSandTDX_areabias_fix.simp0.001.level1_hist_78pfts_CMIP6_simyr2000_HAND_4_col_hillslope_geo_params_nlcd_bedrock.nc'
dssurf = xr.load_dataset(file)

In [35]:
index = dssurf.zbedrock.values < 2
np.nanmedian(met_stats_lse[index,:,0], axis=0)

array([0.46294301, 0.50886277, 0.51978451, 0.52679177, 0.53708631])

In [41]:
met_stats_lse[100,:,0]

array([0.61060931, 0.62978756, 0.63255254, 0.63223743, 0.63592718])

In [42]:
dssurf.zbedrock.values[221]

1.1706964562472866

In [45]:
df_info.iloc[221]

hru_id                                                     3384450
ann_P                                                          0.0
lon_cen                                                  -88.52769
lat_cen                                                   37.53076
AREA                                                 112284035.644
elev_mean                                                191.72973
ave_T                                                          NaN
july_T                                                         NaN
Perimeter                                                    77775
Source                                                       HSTDX
latoutlet                                                 37.47227
lonoutlet                                                -88.54727
areaUSGS                                                    111.13
RawCAMELSarea                                           112.284036
RawTDXarea                                              113.72

In [46]:
pd.read_csv('/glade/work/guoqiang/CTSM_CAMELS/data_paramcailb/ParamCalib_basinparam_3384450.csv')

,Unnamed: 0,Type,Process,PPE_DailyQ_Rank,Parameter,Default_glob,Lower_glob,Upper_glob,Lower_raw,Upper_raw,LowUp_vsPPE,Source,Binding,Method,Default,Lower,Upper
0,0,Hydrology,Infiltration,0,FMAX,3.816694e-01,1.908347e-01,5.725041e-01,1.000000e-01,9.000000e-01,NotInPPE,Surfdata,NaN,Multiplicative,3.434436e-01,1.717218e-01,5.151653e-01
1,1,Hydrology,Infiltration\r\nSoil storage,1,fff,5.000000e-01,2.000000e-02,5.000000e+00,2.000000e-02,5.000000e+00,Same,Param,NaN,Multiplicative,5.000000e-01,2.000000e-02,5.000000e+00
2,2,Hydrology,Soil water drainage,10,hksat_sf,1.000000e+00,1.000000e-01,1.000000e+01,1.000000e-01,1.000000e+01,Same,Param,NaN,Multiplicative,1.000000e+00,1.000000e-01,1.000000e+01
3,3,Hydrology,Soil storage capacity,11,sucsat_sf,1.000000e+00,1.000000e-01,1.000000e+01,1.000000e-01,1.000000e+01,Same,Param,NaN,Multiplicative,1.000000e+00,1.000000e-01,1.000000e+01
4,4,Stomatal resistance and photosynthesis,ET (transpiration),2,medlynslope,4.840380e+00,2.420190e+00,7.260570e+00,3.235949e+00,6.934051e+00,BiggerRange,Param,NaN,Multiplicative,4.840380e+00,2.420190e+00,7.260570e+00
5,5,Hydrology,Infiltration,23,e_ice,6.000000e+00,1.000000e-01,8.000000e+00,1.000000e-01,8.000000e+00,Low0to0.1,Param,NaN,Multiplicative,6.000000e+00,1.000000e-01,8.000000e+00
6,6,Hydrology,Soil storage capacity,9,watsat_sf,1.000000e+00,2.500000e-01,2.000000e+00,8.000000e-01,1.200000e+00,BiggerRange,Param,NaN,Multiplicative,1.000000e+00,2.500000e-01,2.000000e+00
7,7,Snow Processes,Snowpack physics,35,upplim_destruct_metamorph,1.750000e+02,1.000000e+01,5.000000e+02,1.000000e+01,5.000000e+02,BiggerRange_FromYifan,Namelist,NaN,Multiplicative,1.750000e+02,1.000000e+01,5.000000e+02
8,8,Hydrology,Canopy Interception,45,interception_fraction,1.000000e+00,1.000000e-01,1.000000e+00,1.000000e-01,1.000000e+00,Low0.5to0.1,Namelist,NaN,Multiplicative,1.000000e+00,1.000000e-01,1.000000e+00
9,9,Hydrology,Canopy Water,13,maximum_leaf_wetted_fraction,5.000000e-02,1.000000e-02,5.000000e-01,1.000000e-02,5.000000e-01,Same,Namelist,NaN,Multiplicative,5.000000e-02,1.000000e-02,5.000000e-01
